In [1]:
#import json
#with open('/Volumes/Extreme SSD/aligned_image5_labels.splits_updated.json') as f:
#    splits = json.load(f, object_hook=lambda d: {int(k): [int(_v) for _v in v] for k, v in d.items()})
#with open('/Volumes/Extreme SSD/aligned_image5_labels_properties.json',"w") as f:
#    json.dump({
#        "splits": splits,
#        "termination_annotations": {
#            5406: 'disappeared'
#        },
#        "attrs":{}
#    },f,indent=2)
#import pandas as pd
#bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df_updated.csv')
#bboxes_df.to_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df_updated2.csv')

In [2]:
import os
os.environ["NAPARI_ASYNC"] = "1"
#os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [3]:
import pandas as pd
import tensorstore as ts
import json
from copy import deepcopy
spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": "/Volumes/Extreme SSD/aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = '/Volumes/Extreme SSD/aligned_image5_image.zarr'
image = ts.open(image_spec).result()[:,0,:,:]
# bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv')
# display(bboxes_df.head())
# with open('/Volumes/Extreme SSD/aligned_image5_labels.splits.json') as f:
#     splits = json.load(f, object_hook=lambda d: {int(k): [int(_v) for _v in v] for k, v in d.items()})

In [4]:
import numpy as np
import tensorstore as ts
import trackarray_tensorstore as tats
from pathlib import Path

print(labels.shape)
bboxes_df_path = Path('/Volumes/Extreme SSD/aligned_image5_bboxes_df_updated2.csv')
properties_path = Path('/Volumes/Extreme SSD/aligned_image5_labels_properties.json')
ta = tats.read_files(labels, bboxes_df_path, properties_path)

(1965, 19991, 15437)


In [ ]:
if ta.attrs == {}:
    ta.attrs = {
        "verified_track_ids": [62395, 5407, 5406, 983],
        "candidate_track_ids" : [145067, 173436]
    }

In [ ]:
ta.attrs

{'verified_track_ids': [145067, 983, 62395, 173436, 5406, 5407],
 'candidate_track_ids': []}

In [ ]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()

In [ ]:
widget = StateMachineWidget(viewer, ta, image, 
                            verified_track_ids=ta.attrs.get("verified_track_ids", []),
                            candidate_track_ids=ta.attrs.get("candidate_track_ids", []),
                            crop_size=2048)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [10]:
print(widget.verified_track_ids)

{np.uint32(62395), np.uint32(5407), np.uint32(5406), np.uint32(983)}


In [16]:
bboxes_df = tats._trackarr._bbox_dict_to_df(widget.ta._bboxes_dict)

In [20]:
termination_annotations = widget.ta.termination_annotations

In [21]:
termination_annotations

{np.uint32(5406): 'disappeared'}

In [22]:
bboxes_df.to_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df_updated.csv', index=False)

In [23]:
splits = widget.ta.splits

In [24]:
with open('/Volumes/Extreme SSD/aligned_image5_labels.splits_updated.json', 'w') as f:
    json.dump(splits, f)